In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
ob_preci = pd.read_csv(r"G:\learn\python_works\google_ee\rudra\datas\attatchments\Rainfall_Monthly.csv")
fo_preci = pd.read_csv(r"G:\learn\python_works\google_ee\rudra\datas\attatchments\corrected_monthly_precipitation.csv")

In [3]:
print(ob_preci)

                    Date  Precipitation
0    01/01/1982 00:00:00          16.00
1    02/01/1982 00:00:00          39.10
2    03/01/1982 00:00:00          25.70
3    04/01/1982 00:00:00          59.20
4    05/01/1982 00:00:00          75.00
..                   ...            ...
484  08/01/2022 00:00:00         205.04
485  09/01/2022 00:00:00         138.63
486  10/01/2022 00:00:00         132.60
487  11/01/2022 00:00:00           0.00
488  12/01/2022 00:00:00           0.00

[489 rows x 2 columns]


In [25]:
print(fo_preci)

      Year  Month  Precipitation
0     2006      1      15.465209
1     2006      2      12.961373
2     2006      3      11.612478
3     2006      4      74.072916
4     2006      5      84.875134
...    ...    ...            ...
1135  2100      8     203.181778
1136  2100      9     116.533033
1137  2100     10      19.979811
1138  2100     11       9.624789
1139  2100     12      11.330756

[1140 rows x 3 columns]


In [11]:
import pandas as pd
import matplotlib.pyplot as plt

# Set font properties
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 12

# Step 1: Read the CSV files
try:
    ob_preci = pd.read_csv(r"G:\learn\python_works\google_ee\rudra\datas\attatchments\Rainfall_Monthly.csv")
    fo_preci = pd.read_csv(r"G:\learn\python_works\google_ee\rudra\datas\attatchments\corrected_monthly_precipitation.csv")
except FileNotFoundError as e:
    print(f"Error: {e}. Please check the file paths.")
    exit()

# Step 2: Convert 'Date' to datetime and extract Year for observed data
ob_preci['Date'] = pd.to_datetime(ob_preci['Date'], format='%m/%d/%Y %H:%M:%S')
ob_preci['Year'] = ob_preci['Date'].dt.year
ob_preci['Month'] = ob_preci['Date'].dt.month

# Step 3: Baseline data (1982–2011) from observed data
baseline_data = ob_preci[(ob_preci['Year'] >= 1982) & (ob_preci['Year'] <= 2011)]
baseline_monthly_means = baseline_data.groupby('Month')['Precipitation'].mean().reset_index()
baseline_annual_mean = baseline_data['Precipitation'].mean()
print(f"Baseline Annual Mean: {baseline_annual_mean} mm")

# Step 4: Define periods
periods = {
    'Baseline': (1982, 2011),
    '2020s': (2011, 2040),
    '2050s': (2041, 2070),
    '2080s': (2071, 2100)
}

# Step 5: Calculate monthly and annual averages
month_names = [pd.to_datetime(f'2025-{m}-01').strftime('%b') for m in range(1, 13)] + ['Annual']
period_means = {period: [] for period in periods.keys()}

for month in list(range(1, 13)) + [0]:  # Fixed: Convert range to list before concatenation
    if month == 0:
        baseline_mean = baseline_annual_mean
        month_condition = slice(None)
        data_source = ob_preci if 'Baseline' in period_means else fo_preci
    else:
        baseline_mean = baseline_monthly_means[baseline_monthly_means['Month'] == month]['Precipitation'].iloc[0]
        month_condition = (fo_preci['Month'] == month)
        data_source = fo_preci

    for period_name, (start_year, end_year) in periods.items():
        if period_name == 'Baseline':
            period_data = baseline_data[baseline_data['Month'] == month] if month > 0 else baseline_data
        else:
            period_data = data_source[(data_source['Year'] >= start_year) & (data_source['Year'] <= end_year)]
            if month > 0:
                period_data = period_data[period_data['Month'] == month]
        mean_precip = period_data['Precipitation'].mean()
        period_means[period_name].append(mean_precip if not pd.isna(mean_precip) else 0)
        if month == 0:
            print(f"Average {period_name} Annual Precipitation: {mean_precip if not pd.isna(mean_precip) else 'No data'} mm")

# Step 6: Create DataFrame for averages and save to CSV
df_averages = pd.DataFrame({'Month': month_names})
for period_name in periods:
    df_averages[f'Average Precipitation {period_name} (mm)'] = period_means[period_name]
df_averages.to_csv('precipitation_by_period_data.csv', index=False)
print("Plotted data saved as 'precipitation_by_period_data.csv'.")

# Step 7: Calculate relative changes in percentage
relative_changes = {period: [] for period in ['2020s', '2050s', '2080s']}
for month in list(range(1, 13)) + [0]:  # Fixed: Convert range to list before concatenation
    if month == 0:
        baseline_mean = baseline_annual_mean
    else:
        baseline_mean = baseline_monthly_means[baseline_monthly_means['Month'] == month]['Precipitation'].iloc[0]
    
    if pd.isna(baseline_mean):
        for period in ['2020s', '2050s', '2080s']:
            relative_changes[period].append(0)
        continue

    for period_name in ['2020s', '2050s', '2080s']:
        period_mean = period_means[period_name][month - 1 if month > 0 else 12]
        if pd.isna(period_mean):
            relative_changes[period_name].append(0)
        else:
            relative_change = ((period_mean - baseline_mean) / baseline_mean) * 100
            relative_changes[period_name].append(relative_change)

# Step 8: Create DataFrame for relative changes and save to CSV
df_relative = pd.DataFrame({'Month': month_names})
for period_name in ['2020s', '2050s', '2080s']:
    df_relative[f'Relative Change {period_name} (%)'] = relative_changes[period_name]
df_relative.to_csv('relative_change_precipitation_data.csv', index=False)
print("Relative change data saved as 'relative_change_precipitation_data.csv'.")

# Step 9: Plot averages
plt.figure(figsize=(10, 6))
x = range(len(month_names))
for i, period_name in enumerate(periods.keys()):
    plt.bar([x[j] + (i - 1.5) * 0.25 for j in range(len(x))], period_means[period_name], 
            width=0.25, label=f'Average {period_name}', 
            color=['blue', 'black', 'skyblue', 'white'][i], edgecolor='black')

plt.xlabel('Month')
plt.ylabel('Average Precipitation (mm)')
plt.title('Average Precipitation by Month and Period')
plt.xticks(x, month_names, rotation=45)
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)
plt.ylim(0, 400)
plt.tight_layout()
plt.savefig('precipitation_by_period.png', dpi=300, bbox_inches='tight')
plt.close()
print("Chart saved as 'precipitation_by_period.png'.")

# Step 10: Plot relative changes
plt.figure(figsize=(10, 6))
x = range(len(month_names))
plt.bar([i - bar_width for i in x], relative_changes['2020s'], width=0.25, label='2020s', color='black', edgecolor='black')
plt.bar(x, relative_changes['2050s'], width=0.25, label='2050s', color='skyblue', edgecolor='black')
plt.bar([i + bar_width for i in x], relative_changes['2080s'], width=0.25, label='2080s', color='white', edgecolor='black')
bar_width = 0.25

plt.xlabel('Month')
plt.ylabel('Relative Change in Precipitation (%)')
plt.title('Relative Change in Precipitation by Month and Period (Baseline: 1982–2011)')
plt.xticks(x, month_names, rotation=45)
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)
plt.ylim(-50, 200)
plt.tight_layout()
plt.savefig('relative_change_precipitation.png', dpi=300, bbox_inches='tight')
plt.close()
print("Chart saved as 'relative_change_precipitation.png'.")

Baseline Annual Mean: 119.46027777777778 mm
Average Baseline Annual Precipitation: 119.46027777777778 mm
Average 2020s Annual Precipitation: 105.70390070921985 mm
Average 2050s Annual Precipitation: No data mm
Average 2080s Annual Precipitation: No data mm
Plotted data saved as 'precipitation_by_period_data.csv'.
Relative change data saved as 'relative_change_precipitation_data.csv'.
Chart saved as 'precipitation_by_period.png'.
Chart saved as 'relative_change_precipitation.png'.


In [5]:
baseline_monthly_means

Month
1      19.106667
2      28.543333
3      33.213333
4      65.010000
5     133.300000
6     226.093333
7     374.553333
8     289.000000
9     193.596667
10     41.583333
11      8.750000
12     20.773333
Name: Precipitation, dtype: float64

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Set font properties
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 12

# Step 1: Read the CSV files
try:
    ob_preci = pd.read_csv(r"G:\learn\python_works\google_ee\rudra\datas\attatchments\Rainfall_Monthly.csv")
    fo_preci = pd.read_csv(r"G:\learn\python_works\google_ee\rudra\datas\attatchments\corrected_monthly_precipitation.csv")
except FileNotFoundError as e:
    print(f"Error: {e}. Please check the file paths.")
    exit()

# Step 2: Convert 'Date' to datetime and extract Year
ob_preci['Date'] = pd.to_datetime(ob_preci['Date'], format='%m/%d/%Y %H:%M:%S')
ob_preci['Year'] = ob_preci['Date'].dt.year
fo_preci['Year'] = fo_preci['Year']  # Assuming Year is already a column in forecasted data

# Step 3: Aggregate to annual means
annual_ob_preci = ob_preci.groupby('Year')['Precipitation'].mean().reset_index()
annual_fo_preci = fo_preci.groupby('Year')['Precipitation'].mean().reset_index()

# Step 4: Combine observed and forecasted data
annual_data = pd.concat([annual_ob_preci, annual_fo_preci]).drop_duplicates(subset=['Year'], keep='first').sort_values('Year')

# Step 5: Perform linear regression and calculate R²
def linear_regression_and_r2(x, y):
    coeffs = np.polyfit(x, y, 1)
    trend_line = np.poly1d(coeffs)
    y_mean = np.mean(y)
    ss_tot = np.sum((y - y_mean) ** 2)
    ss_res = np.sum((y - trend_line(x)) ** 2)
    r_squared = 1 - (ss_res / ss_tot) if ss_tot != 0 else 0
    return coeffs, r_squared, trend_line

if not annual_data.empty:
    coeffs, r2, trend = linear_regression_and_r2(annual_data['Year'], annual_data['Precipitation'])
    equation = f'y = {coeffs[0]:.4f}x + {coeffs[1]:.4f}\nR² = {r2:.4f}'
else:
    equation = "No data for Precipitation"
    trend = lambda x: np.full_like(x, np.nan)

# Step 6: Create DataFrame and save to CSV
if not annual_data.empty:
    annual_data.to_csv('annual_precipitation_trends_data.csv', index=False)
    print("Plotted data saved as 'annual_precipitation_trends_data.csv'.")
else:
    print("No data available to save to CSV.")

# Step 7: Create the plot
plt.figure(figsize=(10, 6))

# Plot observed and forecasted data with different colors
observed_data = annual_data[(annual_data['Year'] >= 1982) & (annual_data['Year'] <= 2022)]
forecasted_data = annual_data[(annual_data['Year'] >= 2023) & (annual_data['Year'] <= 2100)]
if not observed_data.empty:
    plt.plot(observed_data['Year'], observed_data['Precipitation'], color='blue', label='Observed (1982–2022)', linewidth=2)
if not forecasted_data.empty:
    plt.plot(forecasted_data['Year'], forecasted_data['Precipitation'], color='red', label='Forecasted (2023–2100)', linewidth=2)

# Plot trend line across all data
if not annual_data.empty:
    plt.plot(annual_data['Year'], trend(annual_data['Year']), color='black', linestyle='--', label='Trend Line', linewidth=1)

# Annotations for equation and R²
if not annual_data.empty:
    plt.text(0.02, 0.98, equation, transform=plt.gca().transAxes, verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

# Labels and title
plt.xlabel('Year')
plt.ylabel('Precipitation (mm)')
plt.title('Annual Precipitation Trend with Trend Line (1982–2100)')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)

# Set y-axis range from 0 to 500 mm
plt.ylim(0, 500)

# Adjust layout and save
plt.tight_layout()
plt.savefig('annual_precipitation_trends.png', dpi=300, bbox_inches='tight')
plt.close()
print("Chart saved as 'annual_precipitation_trends.png'.")

Plotted data saved as 'annual_precipitation_trends_data.csv'.
Chart saved as 'annual_precipitation_trends.png'.


In [7]:
import pandas as pd
import matplotlib.pyplot as plt

# Set font properties
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 12

# Step 1: Read the CSV files
try:
    ob_preci = pd.read_csv(r"G:\learn\python_works\google_ee\rudra\datas\attatchments\Rainfall_Monthly.csv")
    fo_preci = pd.read_csv(r"G:\learn\python_works\google_ee\rudra\datas\attatchments\corrected_monthly_precipitation.csv")
except FileNotFoundError as e:
    print(f"Error: {e}. Please check the file paths.")
    exit()

# Step 2: Convert 'Date' to datetime and extract Year
ob_preci['Date'] = pd.to_datetime(ob_preci['Date'], format='%m/%d/%Y %H:%M:%S')
ob_preci['Year'] = ob_preci['Date'].dt.year
fo_preci['Year'] = fo_preci['Year']  # Assuming Year is already a column in forecasted data

# Step 3: Aggregate to annual means
annual_ob_preci = ob_preci.groupby('Year')['Precipitation'].mean().reset_index()
annual_fo_preci = fo_preci.groupby('Year')['Precipitation'].mean().reset_index()

# Step 4: Combine observed and forecasted data
annual_data = pd.concat([annual_ob_preci, annual_fo_preci]).drop_duplicates(subset=['Year'], keep='first').sort_values('Year')

# Step 5: Define periods
periods = {
    'Baseline': (1982, 2011),
    '2020s': (2011, 2040),
    '2050s': (2041, 2070),
    '2080s': (2071, 2100)
}

# Step 6: Calculate average precipitation for each period
period_means = {}
for period_name, (start_year, end_year) in periods.items():
    if period_name == 'Baseline':
        period_data = annual_ob_preci[(annual_ob_preci['Year'] >= start_year) & (annual_ob_preci['Year'] <= end_year)]
    else:
        period_data = annual_fo_preci[(annual_fo_preci['Year'] >= start_year) & (annual_fo_preci['Year'] <= end_year)]
    mean_precip = period_data['Precipitation'].mean()
    period_means[period_name] = mean_precip if not pd.isna(mean_precip) else 0
    print(f"Average {period_name} Precipitation: {mean_precip if not pd.isna(mean_precip) else 'No data'}")

# Step 7: Create DataFrame and save to CSV
df = pd.DataFrame({'Period': list(period_means.keys()), 'Average Precipitation (mm)': list(period_means.values())})
df.to_csv('precipitation_by_period_data.csv', index=False)
print("Plotted data saved as 'precipitation_by_period_data.csv'.")

# Step 8: Create the plot
plt.figure(figsize=(8, 6))
plt.bar(period_means.keys(), period_means.values(), color=['blue', 'black', 'skyblue', 'white'], edgecolor='black')
plt.xlabel('Period')
plt.ylabel('Average Precipitation (mm)')
plt.title('Average Precipitation by Period')
plt.grid(True, linestyle='--', alpha=0.7)

# Set y-axis range from 0 to 400 mm
plt.ylim(0, 400)

# Adjust layout and save
plt.tight_layout()
plt.savefig('precipitation_by_period.png', dpi=300, bbox_inches='tight')
plt.close()
print("Chart saved as 'precipitation_by_period.png'.")

Average Baseline Precipitation: 119.46027777777779
Average 2020s Precipitation: 114.46642469095279
Average 2050s Precipitation: 125.77484328274446
Average 2080s Precipitation: 137.41955872191113
Plotted data saved as 'precipitation_by_period_data.csv'.
Chart saved as 'precipitation_by_period.png'.
